In [86]:
import torch
import torch.nn as nn

In [87]:
import wandb
!wandb login

wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [88]:
from dataclasses import dataclass


@dataclass 
class ModelArgs:
    device = 'cuda'
    no_of_neurons = 128
    block_size = 64
    batch_size = 32
    dropout = 0.1
    epoch = 50
    max_lr = 1e-4

In [89]:
ModelArgs.device

'cuda'

In [90]:
torch.set_default_device(ModelArgs.device)

In [91]:
import torch
from torch.utils.data import Dataset, DataLoader


torch.manual_seed(0)


num_samples = 10000 
seq_length = ModelArgs.block_size  
device = ModelArgs.device  


t = torch.linspace(0, 100, num_samples + seq_length, device=device)
# data = torch.sin(t) + 0.1 * torch.randn_like(t)  
data = t

X_tensor = torch.stack([data[i:i+seq_length] for i in range(num_samples)])
y_tensor = data[seq_length:]  # Next value prediction

train_size = int(0.8 * num_samples)

X_train, y_train = X_tensor[:train_size], y_tensor[:train_size]  
X_val, y_val = X_tensor[train_size:], y_tensor[train_size:]  


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


generator = torch.Generator(device=device)


train_loader = DataLoader(
    train_dataset,
    batch_size=ModelArgs.batch_size,
    shuffle=True,  
    generator=generator,  
    # drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    generator=generator, 
    # drop_last=True,
    batch_size=ModelArgs.batch_size,
    shuffle=True, 
)



In [92]:
class InputGate(nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.it = nn.Linear(in_features= ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
        self.ct_bar = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
        
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=-1)
        it = torch.nn.functional.sigmoid(self.it(x))
        ct_bar = torch.nn.functional.tanh(self.ct_bar(x))
        # out = torch.nn.functional.sigmoid(self.linear(x))
        return it, ct_bar

In [93]:
class OutputGate(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.linear = nn.Linear(in_features= ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [94]:
class ForgetGate(nn.Module):
    
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
        
    def forward(self, x, ht_1):
        # print("Forgot: ", x.shape)
        # print("Forget: ", ht_1.shape)
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [95]:
class LSTMBlock(nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.ip = InputGate(device=device, no_of_neurons=no_of_neurons)
        self.op = OutputGate(device=device, no_of_neurons=no_of_neurons)
        self.forget = ForgetGate(device=device, no_of_neurons=no_of_neurons)
        self.no_of_neurons = no_of_neurons
    def forward(self, x, ht_1, ct_1, outputs=None):
        # print("Block: ", x.shape)
        # print("Block: ", ht_1.shape)
        # print("Block: ", ct_1.shape)
        
        ht_1 = torch.randn(ModelArgs.batch_size, self.no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        ct_1 = torch.randn(ModelArgs.batch_size, self.no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        seq_len = x.shape[1]
        if(outputs == None):
            outputs = []
            for t in range(seq_len):
                xt = x[:, t].unsqueeze(-1)
                ft = self.forget(xt, ht_1) * ct_1
                it, ct_bar = self.ip(xt , ht_1)
                ct_bar_prime = it * ct_bar
                ct = ft * ct_1 + ct_bar_prime
                ht = self.op(xt, ht_1) * torch.nn.functional.tanh(ct)
                outputs.append(ht)
                return ht, ct, torch.stack(outputs, dim=1)

In [96]:
class LSTM(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.block1 = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
        self.block2 = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
        # self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        # self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        self.output = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=out_features, device=device, dtype=torch.float32)
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
        # self.embedding = nn.Embedding()
        
    def forward(self, x):
        # x = 
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        ht, ct = self.block1(x)
        # print(ht.shape)
        # print(ct.shape)
        # ht, ct = self.block2(x, ht, ct)
        ht = self.dropout(ht)
        # print("After: ", ht.shape)
        out = self.output(ht)
        return out

In [97]:
# class DeepLSTM(nn.Module):
#     def __init__(self, device, no_of_neurons, out_features):
#         super().__init__()
#         self.block1 = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
#         self.block2 = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
#         self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
#         self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
#         self.output = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
#         self.dropout = nn.Dropout(p=ModelArgs.dropout)
#         # self.embedding = nn.Embedding()
        
#     def forward(self, x):
#         # x = 
#         # print("LSTM: ",x.shape)
#         # print("LSTM: ", self.ht_1.shape)
#         # print("LSTM: ", self.ct_1.shape)
#         ht, ct = self.block1(x, self.ht_1, self.ct_1)
#         # print(ht.shape)
#         # print(ct.shape)
#         ht, ct = self.block2(x, ht, ct)
#         ht = self.dropout(ht)
#         # print("After: ", ht.shape)
#         out = torch.nn.functional.sigmoid((self.output(ht)))
#         return out

In [98]:
class BiLSTM(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.fwd = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
        self.backward = LSTMBlock(device=device, no_of_neurons=no_of_neurons)
        self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        # self.ht_1_bwd = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        # self.ct_1_bwd = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        self.output = nn.Linear(in_features=2*ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
        # self.embedding = nn.Embedding()
        
    def forward(self, x):
        # x = 
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        ht_fwd, ct, otuputs_bwd = self.fwd(x, self.ht_1, self.ct_1)
        # print(ht.shape)
        # print(ct.shape)
        x_rev = torch.flip(x, dims=[0])
        ht_bwd, ct, outputs_bwd = self.backward(x_rev, self.ht_1, self.ct_1)
        ht = torch.cat([ht_fwd, ht_bwd], axis=-1)
        ht = self.dropout(ht)
        # print("After: ", ht.shape)
        out = self.output(ht)
        return out

In [99]:
model = BiLSTM(device=ModelArgs.device, no_of_neurons=ModelArgs.no_of_neurons, out_features=1)
model = model.to(ModelArgs.device)

In [77]:
x = torch.randn((ModelArgs.batch_size, ModelArgs.block_size))
print("---", x)
x = torch.flip(x, dims=[0])
x

--- tensor([[ 0.2053, -0.3194, -0.2658,  ..., -0.5153,  0.8283,  1.0536],
        [-0.6247, -0.9844,  2.8202,  ..., -0.6800, -0.0589, -0.9113],
        [-1.6264,  1.4028,  0.7094,  ..., -0.9767,  1.2364, -1.1417],
        ...,
        [ 1.5665, -0.1105,  1.4408,  ...,  0.9286,  1.1715,  1.1295],
        [-1.2755,  0.6398, -1.3721,  ...,  0.9117,  0.1014, -0.8110],
        [-0.3983,  1.2490,  0.0061,  ..., -1.4961,  0.0137,  0.7716]],
       device='cuda:0')


tensor([[-0.3983,  1.2490,  0.0061,  ..., -1.4961,  0.0137,  0.7716],
        [-1.2755,  0.6398, -1.3721,  ...,  0.9117,  0.1014, -0.8110],
        [ 1.5665, -0.1105,  1.4408,  ...,  0.9286,  1.1715,  1.1295],
        ...,
        [-1.6264,  1.4028,  0.7094,  ..., -0.9767,  1.2364, -1.1417],
        [-0.6247, -0.9844,  2.8202,  ..., -0.6800, -0.0589, -0.9113],
        [ 0.2053, -0.3194, -0.2658,  ..., -0.5153,  0.8283,  1.0536]],
       device='cuda:0')

In [80]:
!pip install torchinfo

from torchinfo import summary

x = torch.randint(0, 100, (ModelArgs.batch_size,ModelArgs.block_size))  # Random integer between 0 and 100
x = x.to(ModelArgs.device)
summary(model=model,
        input_data=x,
        # input_size=(ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
BiLSTM (BiLSTM)                          [32, 64]             [32, 1]              --                   True
├─LSTMBlock (fwd)                        [32, 64]             [32, 128]            --                   True
│    └─ForgetGate (forget)               [32, 1]              [32, 128]            --                   True
│    │    └─Linear (linear)              [32, 129]            [32, 128]            16,640               True
│    └─InputGate (ip)                    [32, 1]              [32, 128]            --                   True
│    │    └─Linear (it)                  [32, 129]            [32, 128]            16,640               True
│    │    └─Linear (ct_bar)              [32, 129]            [32, 128]            16,640               True
│    └─OutputGate (op)                   [32, 1]              [32, 128]            --                   True
│    │    └─Li

In [84]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=ModelArgs.max_lr)

In [100]:
model.train()
train_losses =  torch.zeros(len(train_loader))
val_losses = torch.zeros(len(val_loader))
wandb.init(
    project='LSTM-From-Scratch'
)
for epoch in range(ModelArgs.epoch):
    
    count = 0
    for X, y in train_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        train_losses[count] = loss.item()
        # print("Loss: ", loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
    # count = 0
    model.eval()
    count = 0
    for X, y in val_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        
        # print("Loss: ", loss.item())
        val_losses[count] = loss.item()
        
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        count += 1
    model.train()
    wandb.log({
      "Train Loss": train_losses.mean(),
      "Val Loss": val_losses.mean(),
      "epoch": epoch  
    })
    print("Epoch: ", epoch, "|", "Train Loss: ", train_losses.mean(),  "|", "Val Loss: ", val_losses.mean())


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 32 for tensor number 1 in the list.